# **Biomolecular Emulator (BioEmu) in ColabFold**
<img src="https://github.com/microsoft/bioemu/raw/main/assets/emu.png" height="130" align="right" style="height:240px">

[BioEmu](https://github.com/microsoft/bioemu) is a framework for emulating biomolecular dynamics and integrating structural prediction tools to accelerate research in structural biology and protein engineering. This notebook uses BioEmu with ColabFold to generate the MSA and identify cluster conformations using Foldseek.



For more details, please read the [BioEmu Preprint](https://www.biorxiv.org/content/10.1101/2024.12.05.626885v2).


## To run
Either run each cell sequentially, or click on `Runtime -> Run All` after choosing the desired sampling config

In [1]:
#@title Sample with following config
#@markdown - `sequence`: Monomer sequence to sample
sequence = "MGCTLSAEDKAAVERSKMIDRNLREDGEKAAREVKLLLLGAGESGKNTIVKQMKIIHEAGYSEEECKQYKAVVYSNTIQSIIAIIRAMGRLKIDFGDSARADDARQLFVLAGAAEEGFMTAELAGVIKRLWKDSGVQACFNRSREYQLNDSAAYYLNDLDRIAQPNYIPTQQDVLRTRVKTTGIVETHFTFKDLHFKMFDVGAQRSERKKWIHCFEGVTAIIFCVALSDYDLVLAEDEEMNRMHASMKLFDSICNNKWFTDTSIILFLNKKDLFEEKIKKSPLTICYPEYAGSNTYEEAAAYIQCQFEDLNKRKDTKEIYTHFTCSTDTKNVQFVFDAVTDVIIKNNLKDCGLF"  #@param {type:"string"}
#@markdown - `num_samples`: Number of samples requested
num_samples = 5  #@param {type:"integer"}
#@markdown - `jobname`: Name assigned to this job
jobname = "d2r_full"  #@param {type:"string"}
#@markdown - `filter_samples`: Whether to filter unphysical samples (e.g., those containing chain breaks) from the written samples
filter_samples = True #@param {type:"boolean"}
# #@param {type:"boolean"}
# ------------------------
# Copied logic from ColabFold
# ------------------------
import os
import re
import hashlib

def add_hash(x, seq):
    """Append a short SHA-1 hash of seq to x."""
    return x + "_" + hashlib.sha1(seq.encode()).hexdigest()[:5]

def folder_is_free(folder):
    """Return True if folder doesn't exist."""
    return not os.path.exists(folder)

jobname_clean = re.sub(r'\W+', '', jobname)
sequence = "".join(sequence.split())
jobname = add_hash(jobname_clean, sequence)

if not folder_is_free(jobname):
    n = 0
    while not folder_is_free(f"{jobname}_{n}"):
        n += 1
    jobname = f"{jobname}_{n}"

output_dir = os.path.join("/content", jobname)
os.makedirs(output_dir, exist_ok=True)


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
#@title Install dependencies
import os
import sys
import subprocess

_marker_file = '/content/.BIOEMU_INSTALLED'

# Check if we need to install
need_install = not os.path.exists(_marker_file)

if need_install:
    print("="*60)
    print("Installing BioEmu and dependencies...")
    print("="*60)
    
    # Install bioemu - let pip handle all dependencies including PyTorch
    print("\n[1/3] Installing bioemu package...")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'bioemu[md]==0.1.6'], check=True)
    
    # Download foldseek
    print("[2/3] Downloading foldseek...")
    if not os.path.exists('/content/foldseek'):
        subprocess.run('wget -q https://mmseqs.com/foldseek/foldseek-linux-avx2.tar.gz && tar xzf foldseek-linux-avx2.tar.gz', shell=True, check=True)
    
    print("[3/3] Installation complete!")
    
    # Create marker
    with open(_marker_file, 'w') as f:
        f.write('installed')
    
    print("\n" + "="*60)
    print("⚠️  IMPORTANT: Restart the runtime now!")
    print("    Go to: Runtime > Restart runtime")
    print("    Then run this cell again.")
    print("="*60)
else:
    print("BioEmu is already installed.")
    
    # Verify the import works
    try:
        import bioemu
        print(f"✓ bioemu version: {bioemu.__version__ if hasattr(bioemu, '__version__') else 'installed'}")
        print(f"✓ bioemu location: {bioemu.__file__}")
    except ImportError as e:
        print(f"✗ Import error: {e}")
        print("Try: Runtime > Restart runtime, then run this cell again")

# Always set environment variables
os.environ['CONDA_PREFIX'] = '/usr'
os.environ['CONDA_PREFIX_1'] = '/usr'
os.environ['CONDA_DEFAULT_ENV'] = 'base'

# Patch the setup.sh for ColabFold (needed for MSA generation)
# Fixed version: properly install into venv and expand glob for patch paths
python_exec = sys.executable

setup_sh_content = f'''#!/bin/bash
set -ex

echo "Setting up colabfold..."
VENV_FOLDER=$1

# Create venv without pip (avoids ensurepip issues on Colab)
{python_exec} -m venv --without-pip ${{VENV_FOLDER}}

# Install pip using get-pip.py
curl -sS https://bootstrap.pypa.io/get-pip.py | ${{VENV_FOLDER}}/bin/python

# Install uv in the venv
${{VENV_FOLDER}}/bin/pip install uv

# Install colabfold INTO THE VENV (not system)
${{VENV_FOLDER}}/bin/uv pip install --python ${{VENV_FOLDER}}/bin/python 'colabfold[alphafold-minus-jax]==1.5.4'
${{VENV_FOLDER}}/bin/uv pip install --python ${{VENV_FOLDER}}/bin/python --force-reinstall "jax[cuda12]==0.4.35" "numpy==1.26.4"
${{VENV_FOLDER}}/bin/uv pip install --python ${{VENV_FOLDER}}/bin/python --force-reinstall \\
    "nvidia-cublas-cu12==12.8.4.1" \\
    "nvidia-cuda-cupti-cu12==12.8.90" \\
    "nvidia-cuda-nvcc-cu12==12.8.93" \\
    "nvidia-cuda-runtime-cu12==12.8.90" \\
    "nvidia-cudnn-cu12==9.8.0.87" \\
    "nvidia-cufft-cu12==11.3.3.83" \\
    "nvidia-cusolver-cu12==11.7.3.90" \\
    "nvidia-cusparse-cu12==12.5.8.93" \\
    "nvidia-nccl-cu12==2.26.2.post1" \\
    "nvidia-nvjitlink-cu12==12.8.93"

# Patch colabfold install
echo "Patching colabfold installation..."
SCRIPT_DIR=$( cd -- "$( dirname -- "${{BASH_SOURCE[0]}}" )" &> /dev/null && pwd )

# Find the actual site-packages directory (expand the glob properly)
SITE_PACKAGES_DIR=$(ls -d ${{VENV_FOLDER}}/lib/python3.*/site-packages 2>/dev/null | head -1)

if [ -z "$SITE_PACKAGES_DIR" ]; then
    echo "ERROR: Could not find site-packages directory"
    exit 1
fi

echo "Site packages directory: $SITE_PACKAGES_DIR"

# Apply patches
patch "$SITE_PACKAGES_DIR/alphafold/model/modules.py" "$SCRIPT_DIR/modules.patch"
patch "$SITE_PACKAGES_DIR/colabfold/batch.py" "$SCRIPT_DIR/batch.patch"

touch ${{VENV_FOLDER}}/.COLABFOLD_PATCHED
echo "Colabfold installation complete!"
'''

try:
    import importlib.util
    spec = importlib.util.find_spec("bioemu")
    if spec and spec.origin:
        setup_sh_path = os.path.join(os.path.dirname(spec.origin), 'colabfold_setup', 'setup.sh')
        if os.path.exists(os.path.dirname(setup_sh_path)):
            with open(setup_sh_path, 'w') as f:
                f.write(setup_sh_content)
            print(f"✓ Patched ColabFold setup script")
except Exception as e:
    print(f"Note: Could not patch setup.sh: {e}")

BioEmu is already installed.
✓ bioemu version: installed
✓ bioemu location: /usr/local/lib/python3.12/dist-packages/bioemu/__init__.py
✓ Patched ColabFold setup script


In [ ]:
# Check ColabFold installation status
import os
import shutil

colabfold_dir = '/root/.bioemu_colabfold'
colabfold_batch = os.path.join(colabfold_dir, 'bin', 'colabfold_batch')
patched_marker = os.path.join(colabfold_dir, '.COLABFOLD_PATCHED')

if os.path.exists(colabfold_batch):
    print(f"✓ ColabFold is installed at {colabfold_dir}")
    if os.path.exists(patched_marker):
        print("✓ ColabFold is fully patched and ready")
    else:
        print("⚠ ColabFold installed but not marked as patched")
        print("  If you see errors, run Cell 4 again to re-patch, then delete this folder and re-run")
elif os.path.exists(colabfold_dir):
    print(f"⚠ ColabFold directory exists but may be incomplete")
    print(f"  Run Cell 4 to fix the installation")
else:
    print("ColabFold not installed yet - it will be installed when you run BioEmu")

ColabFold directory is clean or doesn't exist


In [5]:
! /usr/local/bin/pip 


Usage:   
  pip3 <command> [options]

Commands:
  install                     Install packages.
  download                    Download packages.
  uninstall                   Uninstall packages.
  freeze                      Output installed packages in requirements format.
  inspect                     Inspect the python environment.
  list                        List installed packages.
  show                        Show information about installed packages.
  check                       Verify installed packages have compatible dependencies.
  config                      Manage local and global configuration.
  search                      Search PyPI for packages.
  cache                       Inspect and manage pip's wheel cache.
  index                       Inspect information available from package indexes.
  wheel                       Build wheels from your requirements.
  hash                        Compute hashes of package archives.
  completion                  A helper c

In [6]:
sequence = '/content/rcsb_pdb_7JVR.fasta'
sequence = 'MGCTLSAEDKAAVERSKMIDRNLREDGEKAAREVKLLLLGAGESGKNTIVKQMKIIHEAGYSEEECKQYKAVVYSNTIQSIIAIIRAMGRLKIDFGDSARADDARQLFVLAGAAEEGFMTAELAGVIKRLWKDSGVQACFNRSREYQLNDSAAYYLNDLDRIAQPNYIPTQQDVLRTRVKTTGIVETHFTFKDLHFKMFDVGAQRSERKKWIHCFEGVTAIIFCVALSDYDLVLAEDEEMNRMHASMKLFDSICNNKWFTDTSIILFLNKKDLFEEKIKKSPLTICYPEYAGSNTYEEAAAYIQCQFEDLNKRKDTKEIYTHFTCSTDTKNVQFVFDAVTDVIIKNNLKDCGLFDYKDDDDAKLQTMHHHHHHHHHHHHHHHADLEDNWETLNDNLKVIEKADNAAQVKDALTKMRAAALDAQKATPPKLEDKSPDSPEMKDFRHGFDILVGQIDDALKLANEGKVKEAQAAAEQLKTTRNAYIQKYLASENLYFQGGTMDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIAVIVFGNVLVCMAVSREKALQTTTNYLIVSLAVADLLVATLVMPWVVYLEVVGEWKFSRIHCDIFVTLDVMMCTASILNLCAISIDRYTAVAMPMLYNTRYSSKRRVTVMISIVWVLSFTISCPLLFGLNNADQNECIIANPAFVVYSSIVSFYVPFIVTLLVYIKIYIVLRRRRKRVNTKRSSRAFRAHLRAPLKGNCTHPEDMKLCTVIMKSNGSFPVNRRRVEAARRAQELEMEMLSSTSPPERTRYSPIPPSHHQLTLPDPSHHGLHSTPDSPAKPEKNGHAKDHPKIAKIFEIQTMPNGKTRTSLKTMSRRKLSQQKEKKATQMLAIVLGVFIICWLPFFITHILNIHCDCNIPPVLYSAFTWLGYVNSAVNPIIYTTFNIEFRKAFLKILHCMGSLLQSELDQLRQEAEQLKNQIRDARKACADATLSQITNNIDPVGRIQMRTRRTLRGHLAKIYAMHWGTDSRLLVSASQDGKLIIWDSYTTNKVHAIPLRSSWVMTCAYAPSGNYVACGGLDNICSIYNLKTREGNVRVSRELAGHTGYLSCCRFLDDNQIVTSSGDTTCALWDIETGQQTTTFTGHTGDVMSLSLAPDTRLFVSGACDASAKLWDVREGMCRQTFTGHESDINAICFFPNGNAFATGSDDATCRLFDLRADQELMTYSHDNIICGITSVSFSKSGRLLLAGYDDFNCNVWDALKADRAGVLAGHDNRVSCLGVTDDGMAVATGSWDSFLKIWNGSSMASNNTASIAQARKLVEQLKMEANIDRIKVSKAAADLMAYCEAHAKEDPLLTPVPASENPFREKKFFCAILDVQLVESGGGLVQPGGSRKLSCSASGFAFSSFGMHWVRQAPEKGLEWVAYISSGSGTIYYADTVKGRFTISRDDPKNTLFLQMTSLRSEDTAMYYCVRSIYYYGSSPFDFWGQGTTLTVSSGGGGSGGGGSGGGGSDIVMTQATSSVPVTPGESVSISCRSSKSLLHSNGNTYLYWFLQRPGQSPQLLIYRMSNLASGVPDRFSGSGSGTAFTLTISRLEAEDVGVYYCMQHLEYPLTFGAGTKLELKGSLEVLFQGPAAAHHHHHHHH'
jobname = 'd2r_2'

In [7]:
sequence

'MGCTLSAEDKAAVERSKMIDRNLREDGEKAAREVKLLLLGAGESGKNTIVKQMKIIHEAGYSEEECKQYKAVVYSNTIQSIIAIIRAMGRLKIDFGDSARADDARQLFVLAGAAEEGFMTAELAGVIKRLWKDSGVQACFNRSREYQLNDSAAYYLNDLDRIAQPNYIPTQQDVLRTRVKTTGIVETHFTFKDLHFKMFDVGAQRSERKKWIHCFEGVTAIIFCVALSDYDLVLAEDEEMNRMHASMKLFDSICNNKWFTDTSIILFLNKKDLFEEKIKKSPLTICYPEYAGSNTYEEAAAYIQCQFEDLNKRKDTKEIYTHFTCSTDTKNVQFVFDAVTDVIIKNNLKDCGLFDYKDDDDAKLQTMHHHHHHHHHHHHHHHADLEDNWETLNDNLKVIEKADNAAQVKDALTKMRAAALDAQKATPPKLEDKSPDSPEMKDFRHGFDILVGQIDDALKLANEGKVKEAQAAAEQLKTTRNAYIQKYLASENLYFQGGTMDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIAVIVFGNVLVCMAVSREKALQTTTNYLIVSLAVADLLVATLVMPWVVYLEVVGEWKFSRIHCDIFVTLDVMMCTASILNLCAISIDRYTAVAMPMLYNTRYSSKRRVTVMISIVWVLSFTISCPLLFGLNNADQNECIIANPAFVVYSSIVSFYVPFIVTLLVYIKIYIVLRRRRKRVNTKRSSRAFRAHLRAPLKGNCTHPEDMKLCTVIMKSNGSFPVNRRRVEAARRAQELEMEMLSSTSPPERTRYSPIPPSHHQLTLPDPSHHGLHSTPDSPAKPEKNGHAKDHPKIAKIFEIQTMPNGKTRTSLKTMSRRKLSQQKEKKATQMLAIVLGVFIICWLPFFITHILNIHCDCNIPPVLYSAFTWLGYVNSAVNPIIYTTFNIEFRKAFLKILHCMGSLLQSELDQLRQEAEQLKNQIRDARKACADATLSQITNNIDPVGRIQMRTRRTLR

In [ ]:
#@title Run BioEmu
import os

# Set environment variables
os.environ['CONDA_PREFIX'] = '/usr'
os.environ['CONDA_PREFIX_1'] = '/usr'
os.environ['CONDA_DEFAULT_ENV'] = 'base'

from bioemu.sample import main as sample

print(f"Sequence length: {len(sequence)} residues")

# For very long sequences (>500 residues), filtering often removes all samples
# Disable filtering for long sequences to get results
if len(sequence) > 500:
    print("⚠️ Long sequence detected - disabling filter_samples to avoid empty results")
    use_filter = False
else:
    use_filter = filter_samples

print(f"Using filter_samples={use_filter}")

output_dir = f'/content/{jobname}'
sample(sequence=sequence, num_samples=num_samples, output_dir=output_dir, filter_samples=use_filter)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
INFO:bioemu.get_embeds:Using cached embeddings in /root/.bioemu_embeds_cache.
INFO:bioemu.get_embeds:Using cached embeddings in /root/.bioemu_embeds_cache.
INFO:bioemu.get_embeds:Using cached embeddings in /root/.bioemu_embeds_cache.
INFO:bioemu.get_embeds:Using cached embeddings in /root/.bioemu_embeds_cache.


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# Debug: Check what was generated
import os
from glob import glob

print(f"Output directory: {output_dir}")
if os.path.exists(output_dir):
    files = os.listdir(output_dir)
    print(f"Files in output: {files}")
    
    # Check for .npz batch files (intermediate samples)
    npz_files = glob(os.path.join(output_dir, "*.npz"))
    print(f"NPZ batch files: {len(npz_files)}")
    
    # If there are npz files, check their contents
    if npz_files:
        import numpy as np
        for f in npz_files[:2]:  # Check first 2
            data = np.load(f)
            print(f"  {os.path.basename(f)}: keys={list(data.keys())}")
else:
    print("Output directory doesn't exist yet")

In [ ]:
#@title Write samples and run `foldseek`
#@markdown - `n_write_samples`: Number of samples to randomly select for clustering. Set to `-1` to select all available samples
#@markdown - `tmscore_threshold`: TM-score threshold used for foldseek clustering
#@markdown - `coverage_threshold`: Coverage threshold used for foldseek clustering
#@markdown - `seq_id`: Sequence identity threshold used for foldseek clustering

n_write_samples = -1 #@param {type:"integer"}
tmscore_threshold = 0.6 #@param {type: "number"}
coverage_threshold = 0.7 #@param {type: "number"}
seq_id = 0.95 #@param {type: "number"}

import numpy as np
import mdtraj

_py3dmol_installed_file = '/content/.py3dmol'
if not os.path.exists(_py3dmol_installed_file):
    os.system('uv pip install py3Dmol')
    os.system(f"touch {_py3dmol_installed_file}")

import py3Dmol
pdb_sample_dir = os.path.join('/content', 'pdb_samples')
os.makedirs(pdb_sample_dir, exist_ok=True)

def write_some_samples(topology_file: str, trajectory_file: str, output_dir:str, n_samples: int) -> None:
    traj = mdtraj.load(trajectory_file, top=topology_file)
    assert traj.n_frames >= n_samples
    if n_samples == -1:
        sample_indices = np.arange(traj.n_frames)
    else:
        sample_indices = np.random.choice(np.arange(traj.n_frames), size=n_samples, replace=False)
    for idx in sample_indices:
        traj[idx].save_pdb(os.path.join(output_dir, f'sample_{idx}.pdb'))


topology_file = os.path.join(output_dir, "topology.pdb")
trajectory_file = os.path.join(output_dir, "samples.xtc")

write_some_samples(topology_file=topology_file,
                   trajectory_file=trajectory_file,
                   output_dir=pdb_sample_dir,
                   n_samples=n_write_samples)

# Foldseek
import os
import subprocess
import tempfile

import pandas as pd

def parse_foldseek_cluster_results(cluster_table_path: str) -> dict[int, list[str]]:
    """
    Parses the result of foldseek clustering

    Args:
        cluster_table: path of the output cluster table from foldseek

    Returns:
        Dictionary mapping cluster indices to members

    """

    cluster_table = pd.read_csv(cluster_table_path, sep=r"\s+", header=None)

    cluster_idx_to_members = {}

    for index, group in enumerate(cluster_table.groupby(0)):
        cluster_idx_to_members[index] = sorted(list(group[1][1]))

    return cluster_idx_to_members


def foldseek_cluster(
    input_dir: str,
    out_prefix: str | None = None,
    tmscore_threshold: float = 0.7,
    coverage_threshold: float = 0.9,
    seq_id: float = 0.7,
    coverage_mode: int = 1,
) -> dict[int, set[str]]:
    """
    Runs foldseek easy cluster

    Args:
        input_dir (str): input directory with .cif or .pdb files
        out_prefix (str | None): the prefix of the output files, if None a temporary directory will be used
        tmscore_threshold (float): the tm-score threshold used for clustering
        coverage_threshold (float): the coverage threshold used for clustering
        seq_id (float): the sequence identity threshold used for clustering
        coverage_mode (int): mode used by mmseqs/foldseek to compute coverage

    Returns:
        Dictionary mapping cluster indices to members
    """

    with tempfile.TemporaryDirectory() as temp_dir:

        with tempfile.TemporaryDirectory() as temp_out_dir:
            if out_prefix is None:
                out_prefix = os.path.join(temp_out_dir, "output")

            res = subprocess.run(
                "/content/foldseek/bin/foldseek easy-cluster "
                + input_dir
                + " "
                + out_prefix
                + " "
                + temp_dir
                + " -c  "
                + str(coverage_threshold)
                + " --min-seq-id "
                + str(seq_id)
                + " --tmscore-threshold "
                + str(tmscore_threshold)
                + " --cov-mode "
                + str(coverage_mode)
                + " --single-step-clustering",
                shell=True,
            )
            assert res.returncode == 0, "Something went wrong with foldseek"

            cluster_idx_to_members = parse_foldseek_cluster_results(out_prefix + "_cluster.tsv")

    return cluster_idx_to_members

!chmod +x '/content/foldseek/bin/foldseek'

# Get foldseek clusters
clusters = foldseek_cluster(input_dir=pdb_sample_dir, tmscore_threshold=tmscore_threshold,
                            coverage_threshold=coverage_threshold, seq_id=seq_id)
n_clusters = len(clusters)
print(f'{n_clusters} clusters detected')

# Write foldseek clusters to output dir
import json

with open(os.path.join(output_dir, 'foldseek_clusters.json'), 'w') as json_handle:
    json.dump(clusters, json_handle)


# Write XTC with one sample per cluster only
cluster_trajs = []
for _cluster_idx, samples in clusters.items():
    sample = list(samples)[0] # Choose first sample in cluster
    pdb_file = os.path.join(pdb_sample_dir, f"{sample}.pdb")
    traj = mdtraj.load_pdb(pdb_file)
    cluster_trajs.append(traj)
joint_traj = mdtraj.join(cluster_trajs)
cluster_topology_file = os.path.join(output_dir, "clustered_topology.pdb")
cluster_trajectory_file = os.path.join(output_dir, "clustered_samples.xtc")
joint_traj[0].save_pdb(cluster_topology_file)
joint_traj.save_xtc(cluster_trajectory_file)


In [ ]:
#@title Display structure
import os
import ipywidgets as widgets
import py3Dmol
from IPython.display import display, clear_output

# Create interactive widgets for cluster and sample selection.
cluster_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=n_clusters - 1,
    step=1,
    description='Cluster No:',
    continuous_update=False
)
sample_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=0,  # will update based on the selected cluster
    step=1,
    description='Sample Idx:',
    continuous_update=False
)
display(cluster_slider, sample_slider)

# Function to visualize a PDB file using py3Dmol.
def show_pdb(pdb_file: str, show_sidechains: bool = False, show_mainchains: bool = True):
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    try:
        with open(pdb_file, 'r') as f:
            pdb_content = f.read()
    except FileNotFoundError:
        print(f"File not found: {pdb_file}")
        return None
    view.addModel(pdb_content, 'pdb')
    view.setStyle({'cartoon': {'color': 'spectrum'}})

    if show_sidechains:
        BB = ['C', 'O', 'N']
        view.addStyle({'and': [{'resn': ["GLY", "PRO"], 'invert': True}, {'atom': BB, 'invert': True}]},
                      {'stick': {'colorscheme': "WhiteCarbon", 'radius': 0.3}})
        view.addStyle({'and': [{'resn': "GLY"}, {'atom': 'CA'}]},
                      {'sphere': {'colorscheme': "WhiteCarbon", 'radius': 0.3}})
        view.addStyle({'and': [{'resn': "PRO"}, {'atom': ['C', 'O'], 'invert': True}]},
                      {'stick': {'colorscheme': "WhiteCarbon", 'radius': 0.3}})
    if show_mainchains:
        BB = ['C', 'O', 'N', 'CA']
        view.addStyle({'atom': BB}, {'stick': {'colorscheme': "WhiteCarbon", 'radius': 0.3}})

    view.zoomTo()
    return view

# Helper to update the sample slider's maximum value based on the selected cluster.
def update_sample_slider(cluster_no):
    available_samples = list(clusters[cluster_no])
    sample_slider.max = max(len(available_samples) - 1, 0)
    # Reset sample_slider's value if it's out of range.
    if sample_slider.value > sample_slider.max:
        sample_slider.value = 0

# Main function to update the viewer whenever widget values change.
def update_view(change=None):
    cluster_no = cluster_slider.value
    update_sample_slider(cluster_no)
    available_samples = list(clusters[cluster_no])
    sample_idx = sample_slider.value

    clear_output(wait=True)
    display(cluster_slider, sample_slider)

    if sample_idx >= len(available_samples):
        print(f"Only {len(available_samples)} samples available in cluster {cluster_no}")
        return

    chosen_sample = available_samples[sample_idx]
    pdb_path = os.path.join("pdb_samples", f"{chosen_sample}.pdb")

    # Check if the file exists before attempting to open it.
    if not os.path.exists(pdb_path):
        print(f"File not found: {pdb_path}")
        return

    print(f"Displaying sample {sample_idx} from cluster {cluster_no}")
    view = show_pdb(pdb_path)
    if view:
        view.show()

# Observe changes to the slider values.
cluster_slider.observe(update_view, names='value')
sample_slider.observe(update_view, names='value')

# Trigger an initial update.
update_view()


In [ ]:
#@title (Optional) Reconstruct sidechains + Run MD relaxation
#@markdown - `reconstruct_sidechains`: whether to reconstruct sidechains via `hpacker`
#@markdown - `run_md`: check to run MD after sidechain reconstruction, otherwise only sidechain reconstruction is performed
#@markdown - `md_protocol`: `LOCAL_MINIMIZATION`: fast but only resolves local problems ; `NVT_EQUIL`: slow but might resolve more severe issues
#@markdown - `one_per_cluster`: Reconstruct sidechains / optionally run MD for only one sample within each foldseek cluster

#@markdown **WARNING**: this step can be quite expensive depending on how many samples you have requested / sequence length. You may want to check the `one_per_cluster` option.

reconstruct_sidechains = False #@param {type: "boolean"}
run_md = True #@param {type:"boolean"}
one_per_cluster = True #@param {type:"boolean"}
md_protocol = "LOCAL_MINIMIZATION" #@param ["LOCAL_MINIMIZATION", "NVT_EQUIL"] {type:"string"}
import bioemu.sidechain_relax
bioemu.sidechain_relax.HPACKER_PYTHONBIN = '/usr/local/envs/hpacker/bin/python'

from bioemu.sidechain_relax import main as sidechainrelax
from bioemu.sidechain_relax import MDProtocol
md_protocol = MDProtocol[md_protocol]
os.environ['CONDA_PREFIX_1'] = '/usr/local/'

if one_per_cluster:
    topology_file = cluster_topology_file
    trajectory_file = cluster_trajectory_file

prefix = 'hpacker-openmm'
if reconstruct_sidechains:
    relaxed_dir = os.path.join(output_dir, prefix)
    os.makedirs(relaxed_dir, exist_ok=True)
    sidechainrelax(pdb_path=topology_file, xtc_path=trajectory_file,
                  outpath=relaxed_dir, prefix=prefix, md_protocol=md_protocol,
                  md_equil=run_md)
    if run_md:
        os.system(f'touch {relaxed_dir}/.RELAXED')


In [ ]:
#@title Package and download results
from google.colab import files
import tarfile
from glob import glob


# Delete bioemu .npz batch files
npz_files = glob(os.path.join(output_dir, "*.npz"))
[os.unlink(npz) for npz in npz_files]

# Add sidechain reconstruction files to output (#82)
import shutil
sidechain_topology = '/content/hpacker-openmm_sidechain_rec.pdb'
sidechain_trajectory = '/content/hpacker-openmm_sidechain_rec.xtc'

if os.path.exists(sidechain_topology) and os.path.exists(sidechain_trajectory):
    shutil.copyfile(sidechain_topology, os.path.join(relaxed_dir, os.path.basename(sidechain_topology)))
    shutil.copyfile(sidechain_trajectory, os.path.join(relaxed_dir, os.path.basename(sidechain_trajectory)))

# Add small README
README = """
# BioEmu Colab output:

`samples.xtc` and `topology.pdb`: Trajectory and topology files of all drawn samples.
`cluster_samples.xtc` and `cluster_topology.pdb`: Trajectory and topology files of clustered samples via
                      foldseek using the parameters specified in the notebook.
`foldseek_clusters.json`: Foldseek cluster assignment of all drawn samples
`sequence.fasta`: FASTA file containing the sequence that was sampled
`hpacker-openmm/`
    |- `hpacker-openmm_sidechain_rec.pdb` and `hpacker-openmm_sidechain_rec.xtc`: Contain sidechain
                      reconstructed samples via `hpacker`.
    |- `hpacker-openmm_md_equil.pdb` and `hpacker-openmm_md_equil.xtc`: Contain MD-equilibrated samples
                      after sidechain reconstruction.

For issues, please visit the [`bioemu` GitHub repository](https://github.com/microsoft/bioemu)

"""

with open(os.path.join(output_dir, "README.md"), "w") as readme_handle:
    readme_handle.write(README)

citations = {
"Lewis2024": """@article{Lewis2024,
author =  {Lewis, Sarah and Hempel, Tim and Jim{\'e}nez-Luna, Jos{\'e} and Gastegger, Michael and Xie, Yu and Foong, Andrew Y. K. and Satorras, Victor Garc{\'\i}a and Abdin, Osama and Veeling, Bastiaan S. and Zaporozhets, Iryna and Chen, Yaoyi and Yang, Soojung and Schneuing, Arne and Nigam, Jigyasa and Barbero, Federico and Stimper, Vincent and Campbell, Andrew and Yim, Jason and Lienen, Marten and Shi, Yu and Zheng, Shuxin and Schulz, Hannes and Munir, Usman and Tomioka, Ryota and Clementi, Cecilia and No{\'e}, Frank},
doi = {10.1101/2024.12.05.626885},
journal = {bioRxiv},
title = {{Scalable emulation of protein equilibrium ensembles with generative deep learning}},
year = {2025},
comment = {BioEmu prediction}
}""",

"Mirdita2021": """@article{Mirdita2022,
author= {Mirdita, Milot and Schütze, Konstantin and Moriwaki, Yoshitaka and Heo, Lim and Ovchinnikov, Sergey and Steinegger, Martin },
doi = {10.1038/s41592-022-01488-1},
journal = {Nature Methods},
title = {{ColabFold: Making Protein folding accessible to all}},
year = {2022},
comment = {ColabFold MMseqs2 MSA server}
}""",
"VanKempen2023": """@article{VanKempen2023,
author = {van Kempen, Michel and Kim, Stephanie S and Tumescheit, Charlotte and Mirdita, Milot and Lee, Jeongjae and Gilchrist, Cameron L M and S{\"{o}}ding, Johannes and Steinegger, Martin},
doi = {10.1038/s41587-023-01773-0},
journal = {Nature Biotechnology},
title = {{Fast and accurate protein structure search with Foldseek}},
year = {2023},
comment = {Clustering structures}
}""",
}

from pathlib import Path

def write_bibtex(
    result_dir: Path,
    bibtex_file: str = "cite.bibtex",
) -> Path:
    to_cite = ["Lewis2024"]
    to_cite += ["Mirdita2021"]
    to_cite += ["VanKempen2023"]

    bibtex_file = result_dir.joinpath(bibtex_file)
    with bibtex_file.open("w", encoding="utf-8") as writer:
        for i in to_cite:
            writer.write(citations[i])
            writer.write("\n")

    print(f"Found {len(to_cite)} citations for tools or databases")
    return bibtex_file

write_bibtex(Path(output_dir))

output_tarfile = f'/content/{jobname}.tar.gz'
with tarfile.open(output_tarfile, 'w:gz') as tar_handle:
    tar_handle.add(name=f'{output_dir}', arcname=os.path.basename(output_dir), recursive=True)

files.download(output_tarfile)